В реальных задачах, как правило, мы не имеем готовой к использованию таблицы, а получаем данные из разных источников, и очень часто приходится объединять несколько таблиц в одну. 

Например, при построении воронки продаж для сквозной аналитики, обогащении выгрузки данными другой системы, сравнении полноты данных в системах и при других задачах.

При этом требования к объемам данных постоянно растут. Если у вас нет под рукой настроенной базы данных или личного разработчика, то объединить даже «небольшие» таблицы в пару миллионов строк по визитам на сайт и покупкам в CRM станет проблемой, а по современным меркам это сравнительно небольшие таблицы. При этом объединять их надо сразу по нескольким измерениям. Например, чтобы получить конверсию в продажи на каждый день в разбивке по рекламному источнику и региону.

Наша задача — посчитать конверсию из визитов в покупки в разрезе источников трафика.

Т. е. для каждой пары значений source и medium нам нужно взять количество визитов и покупок и совместить их в одной таблице. Для тех, кто знаком с SQL, это аналог операции JOIN. На первом шаге нам понадобится самый простой случай объединения этих файлов по одному столбцу (источнику).

visits_by_source.txt — в первом столбце стоит источник трафика, во втором — сумма визитов:

burgerclub 1197
city-magazine 528
facebook 3144
...
orders_by_source.txt — в первом столбце стоит источник трафика, во втором — количество покупок, в третьем — суммарная стоимость покупок:

burgerclub 10 185
city-magazine 5 81
direct 5 88
...

Наша задача будет состоять в следующем: нужно написать функцию, на вход которой будем подавать очередную строчку из файла visits_by_source.txt. В ответ должны получать количество покупок из файла orders_by_source.txt, которое соответствует этой строчке. Т. е. в самом простом случае в файле orders_by_source.txt надо найти строчку с таким же источником и вернуть значение из второго столбца.

In [11]:
orders_dict = {}
with open('orders_by_source.txt', 'r') as f:
        for line in f:
            line = line.strip().split('\t') 
            orders_dict[line[0]]=int(line[1])

In [12]:
line

['yandex', '104', '1818']

In [13]:
orders_dict

{'burgerclub': 10,
 'city-magazine': 5,
 'direct': 5,
 'facebook': 5,
 'food-delivery': 10,
 'foody': 3,
 'google': 77,
 'newsletter': 5,
 'promo': 68,
 'vk': 2,
 'yandex': 104}

In [14]:

def searchForLine( source, orders_dict ):
    """Функция по названию источника source ищет соответствующую строку в файле orders_by_source.txt.
      Возвращает количество покупок, соответствующее источнику source. Если источник не найден, то возвращает 0"""
  
    if source in orders_dict:
            return int( orders_dict[source] )
    

In [15]:
searchForLine('burgerclub', orders_dict)

10

Напишите цикл, в котором в каждую строчку файла visits_by_source.txt подставляется соответствующее количество покупок orders. Т. е. цикл заканчивается следующей строкой:

print(source, visits, orders, orders / visits)

Выведите на экран конверсию визитов в покупки для каждого источника (т. е. отношение orders к visits). Какое значение конверсии у источника vk? Ответ округлите до тысячных (т. е. ответ будет иметь формат 0.009).

In [19]:
       with open('visits_by_source.txt', 'r') as f:
        for line in f:
            line = line.strip().split('\t')
            source = line[0]
            visits = int(line[1])
            orders = searchForLine( source, orders_dict )
            print(' Источник {},визитов {},заказов {}, конверсия {:.3f}' .format(source,visits, orders, orders / visits))

 Источник burgerclub,визитов 1197,заказов 10, конверсия 0.008
 Источник city-magazine,визитов 528,заказов 5, конверсия 0.009
 Источник facebook,визитов 3144,заказов 5, конверсия 0.002
 Источник food-delivery,визитов 1184,заказов 10, конверсия 0.008
 Источник foody,визитов 421,заказов 3, конверсия 0.007
 Источник google,визитов 10961,заказов 77, конверсия 0.007
 Источник newsletter,визитов 637,заказов 5, конверсия 0.008
 Источник promo,визитов 7405,заказов 68, конверсия 0.009
 Источник vk,визитов 256,заказов 2, конверсия 0.008
 Источник yandex,визитов 11757,заказов 104, конверсия 0.009
 Источник direct,визитов 2156,заказов 5, конверсия 0.002


Т. к. файл visits_by_source.txt может быть очень большим, выводить результаты на экран не особо практично.

Давайте запишем результат в файл.

Для этого мы будем «открывать» файл joined_by_source.txt с параметром 'w' вместо 'r'. При каждом таком «открытии» файл joined_by_source.txt будет создаваться заново (т. е., если он существовал ранее, все его содержимое будет удалено).

Один файл (visits_by_source.txt) у нас уже открыт как переменная f. Соответственно, при открытии файла joined_by_source.txt надо давать ему другое имя (например, f_joined). И вместо функции print пишем команду записи в файл f_joined.write(...).

Обратите внимание, что при записи строки в файл надо указывать символ переноса строки \n в конце каждой строчки, иначе все строки будут записаны как одна:

In [28]:
with open('joined_by_source.txt', 'w') as f_joined:
    with open('visits_by_source.txt', 'r') as f:
        for line in f:
            line = line.strip().split('\t')
            source = line[0]
            visits = int(line[1])
            orders = searchForLine( source, orders_dict )
            f_joined.write('{}\t{}\t{}\n'.format( source, visits, orders) )

In [32]:
with open('joined_by_source.txt', 'r') as f_joined:
     for line in f_joined:
        line = line.strip().split('\t')   
        print(line)

['burgerclub', '1197', '10']
['city-magazine', '528', '5']
['facebook', '3144', '5']
['food-delivery', '1184', '10']
['foody', '421', '3']
['google', '10961', '77']
['newsletter', '637', '5']
['promo', '7405', '68']
['vk', '256', '2']
['yandex', '11757', '104']
['direct', '2156', '5']


In [43]:
with open('joined_by_source.txt', 'w') as f_joined:
    f_joined.write('source\tvisits\torders\tconversation\n')
    with open('visits_by_source.txt', 'r') as f:
        for line in f:
            line = line.strip().split('\t')
            source = line[0]
            visits = int(line[1])
            orders = searchForLine( source, orders_dict )
            f_joined.write('{}\t{}\t{}\t{:.3f}\n'.format( source, visits, orders, orders/visits) )

In [44]:
with open('joined_by_source.txt', 'r') as f_joined:
    for line in f_joined:
            line = line.strip().split('\t')
            print(line)

['source', 'visits', 'orders', 'conversation']
['burgerclub', '1197', '10', '0.008']
['city-magazine', '528', '5', '0.009']
['facebook', '3144', '5', '0.002']
['food-delivery', '1184', '10', '0.008']
['foody', '421', '3', '0.007']
['google', '10961', '77', '0.007']
['newsletter', '637', '5', '0.008']
['promo', '7405', '68', '0.009']
['vk', '256', '2', '0.008']
['yandex', '11757', '104', '0.009']
['direct', '2156', '5', '0.002']


In [45]:
import pandas

df = pandas.read_table('joined_by_source.txt')

In [46]:
df

,source,visits,orders,conversation
0,burgerclub,1197,10,0.008
1,city-magazine,528,5,0.009
2,facebook,3144,5,0.002
3,food-delivery,1184,10,0.008
4,foody,421,3,0.007
5,google,10961,77,0.007
6,newsletter,637,5,0.008
7,promo,7405,68,0.009
8,vk,256,2,0.008
9,yandex,11757,104,0.009


In [48]:
df.conversation.mean()

0.007000000000000001